In [9]:
import os
import preprocessing

from gensim import corpora

In [13]:
#texts = preprocessing.corpus_tokens()
#dictionary = corpora.Dictionary(texts)
flat_metadata = preprocessing.flat_unique()
summaries = [paper['summary'].replace('\n', ' ') for paper in flat_metadata]

# Sample
summaries[0][:200]

'Time series forecasting is a crucial component of many important applications, ranging from forecasting the stock markets to energy load prediction. The high-dimensionality, velocity and variety of th'

In [35]:
import spacy
from spacy.lang.en.examples import sentences

nlp = spacy.load('en_core_web_md')

In [41]:
for i in range(len(summaries)):
    doc = nlp(summaries[i])
    for entity in doc.ents:
        print(entity.text, entity.label_)
        
    if i > 1:
        break

Temporal Logistic Neural ORG
BoF ORG
BoF GPE
BoF GPE
more than 4 million CARDINAL
Gaussian ORG
Davis PERSON
1984 DATE
one CARDINAL
Monte Carlo ORG
Monte Carlo ORG
PDMP ORG
Monte Carlo ORG


In [ ]:
sim = []
for i in range(1, len(summaries)):
    # Determine semantic similarities
    doc1 = nlp(summaries[i-1])
    doc2 = nlp(summaries[i])
    similarity = doc1.similarity(doc2)
    #print(doc1.text[:50], doc2.text[:50], similarity)
    sim.append(similarity)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(sim)
plt.show()